In [ ]:
!pip install pyfinite
!pip install sympy

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import numpy as np
import random
import sympy as sp
from numpy.linalg import matrix_rank
from pyfinite import ffield

# Setting display options for numpy
np.set_printoptions(threshold=np.inf)

# Character to binary mapping for encoding
binary_to_char = {
    '0000': 'f', '0001': 'g', '0010': 'h', '0011': 'i',
    '0100': 'j', '0101': 'k', '0110': 'l', '0111': 'm',
    '1000': 'n', '1001': 'o', '1010': 'p', '1011': 'q',
    '1100': 'r', '1101': 's', '1110': 't', '1111': 'u'
}

# Converts an 8-bit number to a string based on the mapping
def byte_to_string(byte):
    binary_str = '{:08b}'.format(byte)
    return binary_to_char[binary_str[:4]] + binary_to_char[binary_str[4:]]

# Maps two characters to a byte
def string_to_hex(chars):
    return chr(16 * (ord(chars[0]) - ord('f')) + (ord(chars[1]) - ord('f')))

# Converts a block of text into hex characters
def text_to_bytes(text):
    hex_string = ""
    for i in range(0, len(text), 2):
        hex_string += string_to_hex(text[i:i+2])
    return hex_string

# Lookup for exponentiation operations to avoid repeated calculations
exponentiation_cache = [[-1]*128 for _ in range(128)]
F7 = ffield.FField(7)

# Bitwise addition using XOR
def bitwise_add(x, y):
    return x ^ y

# Multiplication over finite field
def field_multiply(x, y):
    return F7.Multiply(x, y)

# Exponentiation over finite field with caching
def field_exponent(base, exponent):
    if exponentiation_cache[base][exponent] != -1:
        return exponentiation_cache[base][exponent]

    if exponent == 0:
        result = 1
    elif exponent == 1:
        result = base
    elif exponent % 2 == 0:
        temp = field_exponent(base, exponent // 2)
        result = field_multiply(temp, temp)
    else:
        temp = field_exponent(base, exponent // 2)
        result = field_multiply(field_multiply(temp, temp), base)

    exponentiation_cache[base][exponent] = result
    return result

# Vector addition
def vector_add(v1, v2):
    return [bitwise_add(a, b) for a, b in zip(v1, v2)]

# Scalar multiplication over a vector
def vector_scalar_mult(vector, scalar):
    return [field_multiply(num, scalar) for num in vector]

# Applies a linear transformation over elements
def linear_transform(matrix, elements):
    transformed = [0] * 8
    for row, value in zip(matrix, elements):
        transformed = vector_add(vector_scalar_mult(row, value), transformed)
    return transformed


In [ ]:
# List for storing possible exponents
exponent_options = [[] for _ in range(8)]
# List for storing possible diagonal matrix values
matrix_values = [[[] for _ in range(8)] for _ in range(8)]

# Open input and output files
with open("input_strings.txt", 'r') as input_data, open("output_strings.txt", 'r') as output_data:
    for index, (input_line, output_line) in enumerate(zip(input_data.readlines(), output_data.readlines())):
        input_hex = []
        output_hex = []
        # Convert input and output lines to hex values
        for hex_in in input_line.strip().split():
            input_hex.append(text_to_bytes(hex_in)[index])
        for hex_out in output_line.strip().split():
            output_hex.append(text_to_bytes(hex_out)[index])

        for exp in range(1, 127):
            for diag in range(1, 128):
                valid = True
                for inp_hex, out_hex in zip(input_hex, output_hex):
                    if ord(out_hex) != field_exponent(field_multiply(field_exponent(field_multiply(field_exponent(ord(inp_hex), exp), diag), exp), diag), exp):
                        valid = False
                        break
                if valid:
                    exponent_options[index].append(exp)
                    matrix_values[index][index].append(diag)

print(matrix_values)
print(exponent_options)


In [ ]:

with open("input_strings.txt", 'r') as input_data, open("output_strings.txt", 'r') as output_data:
    for index, (input_line, output_line) in enumerate(zip(input_data.readlines(), output_data.readlines())):
        if index > 6:
            break
        input_hex = []
        output_hex = []
        for hex_in in input_line.strip().split():
            input_hex.append(text_to_bytes(hex_in)[index])
        for hex_out in output_line.strip().split():
            output_hex.append(text_to_bytes(hex_out)[index + 1])

        for exp in range(1, 128):
            for previous_exp, diag_value in zip(exponent_options[index + 1], matrix_values[index + 1][index + 1]):
                for current_exp, current_diag in zip(exponent_options[index], matrix_values[index][index]):
                    valid = True
                    for inp_hex, out_hex in zip(input_hex, output_hex):
                        combined = field_exponent(bitwise_add(field_multiply(field_exponent(field_multiply(field_exponent(ord(inp_hex), current_exp), current_diag), current_exp), exp),
                                                              field_multiply(field_exponent(field_multiply(field_exponent(ord(inp_hex), current_exp), exp), previous_exp), diag_value)), previous_exp)
                        if ord(out_hex) != combined:
                            valid = False
                            break
                    if valid:
                        exponent_options[index + 1] = [previous_exp]
                        matrix_values[index + 1][index + 1] = [diag_value]
                        exponent_options[index] = [current_exp]
                        matrix_values[index][index] = [current_diag]
                        matrix_values[index][index + 1] = [exp]

print(matrix_values)
print(exponent_options)


In [ ]:
# Function to simulate the EAEAE encryption scheme
def simulate_EAEAE(plaintext, linear_matrix, exponent_matrix):
    # Convert plaintext to their ASCII values
    ascii_values = [ord(char) for char in plaintext]
    processed_output = [[0] * 8 for _ in range(5)]

    # First exponentiation step
    for index, value in enumerate(ascii_values):
        processed_output[0][index] = field_exponent(value, exponent_matrix[index])

    # First linear transformation step
    processed_output[1] = linear_transform(linear_matrix, processed_output[0])

    # Second exponentiation step
    for index, value in enumerate(processed_output[1]):
        processed_output[2][index] = field_exponent(value, exponent_matrix[index])

    # Second linear transformation step
    processed_output[3] = linear_transform(linear_matrix, processed_output[2])

    # Final exponentiation step
    for index, value in enumerate(processed_output[3]):
        processed_output[4][index] = field_exponent(value, exponent_matrix[index])

    return processed_output[4]


In [ ]:
# Revised code to iterate and find suitable linear transformations and exponents
for idx in range(6):
    offset = idx + 2
    
    # Extracting the first element from each list in poss_exp
    exponent_list = [e[0] for e in exponent_options if e]
    
    # Preparing the linear transformation matrix and filling it with initial values
    linear_transform_matrix = [[0] * 8 for _ in range(8)]
    for i in range(8):
        for j in range(8):
            linear_transform_matrix[i][j] = 0 if not matrix_values[i][j] else matrix_values[i][j][0]
    
    # Open input and output files
    with open("input_strings.txt", 'r') as infile, open("output_strings.txt", 'r') as outfile:
        for ind, (input_line, output_line) in enumerate(zip(infile.readlines(), outfile.readlines())):
            if ind > (7 - offset):
                continue
            input_hex = [text_to_bytes(hex_value) for hex_value in input_line.strip().split()]
            output_hex = [text_to_bytes(hex_value) for hex_value in output_line.strip().split()]
            
            # Iterate to find the matching matrix element that satisfies the EAEAE transformation
            for potential_value in range(1, 128):
                linear_transform_matrix[ind][ind + offset] = potential_value
                match_found = True
                for input_val, output_val in zip(input_hex, output_hex):
                    if ord(output_val[ind + offset]) != simulate_EAEAE(input_val, linear_transform_matrix, exponent_list)[ind + offset]:
                        match_found = False
                        break
                if match_found:
                    matrix_values[ind][ind + offset] = [potential_value]
                    break

# Update the linear transformation matrix with final values
final_linear_transform_matrix = [[0] * 8 for _ in range(8)]
for i in range(8):
    for j in range(8):
        final_linear_transform_matrix[i][j] = 0 if not matrix_values[i][j] else matrix_values[i][j][0]

print(final_linear_transform_matrix)
print(exponent_list)


In [ ]:
# print(matrix_values)
# print(exponent_options)

In [ ]:
# password = 'lhjkfghjjkilgriqmrkrfghhkjljliif'

# Two halves of the password
password_part_one = 'lhjkfghjjkilgriq'
password_part_two = 'mrkrfghhkjljliif'

# Function to decrypt a password half based on the EAEAE encryption scheme
def decrypt_password_half(password_half):
    # Convert the password part to byte values
    byte_values = text_to_bytes(password_half)
    result = ""
    for index in range(8):
        for candidate in range(128):
            # Generate input for EAEAE transformation
            test_input = result + byte_to_string(candidate) + (16 - len(result) - 2) * 'f'
            # Perform the EAEAE transformation and check if the output matches the byte of the password
            if ord(byte_values[index]) == simulate_EAEAE(text_to_bytes(test_input), final_linear_transform_matrix, exponent_list)[index]:
                result += byte_to_string(candidate)
                break
    return result

# Decrypt both halves of the password and concatenate their byte forms
decrypted_half_one = decrypt_password_half(password_part_one)
decrypted_half_two = decrypt_password_half(password_part_two)
full_decrypted_password = text_to_bytes(decrypted_half_one) + text_to_bytes(decrypted_half_two)

print("Decrypted password: ",full_decrypted_password)


In [ ]:
print("Original input password: ",full_decrypted_password[0:10])